# Linear regression using tensorflow - quick tutorial using the Machine learning code template

In [1]:
import tensorflow as tf
import os
import numpy as np

# Below are the function for the supervised learning workflow

In [2]:
#initialize model parameters
W = tf.Variable(tf.zeros([2,1]),name="weights")
b = tf.Variable(0.,name="bias")

In [3]:
def inference(X):
    return tf.matmul(X,W)+b

In [4]:
def loss(X,Y):
    Y_predicted=inference(X)
    return tf.reduce_sum(tf.squared_difference(Y,Y_predicted))

In [5]:
def inputs():
    weight_age=[[84,46],[73,20],[65,52],[70,30],[76,57],[69,25],[63,28],[72,36]]
    blood_fat_content = [354,190,405,263,451,302,288,385]
    
    return tf.to_float(weight_age),tf.to_float(blood_fat_content)

In [6]:
def train(total_loss):
    learning_rate = 0.0000001
    return	tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [7]:
def evaluate(sess,X,Y):
    print sess.run(inference([[80.,25.]]))#~303
    print sess.run(inference([[65.,25.]]))#~256

# Last, the run function for the model

In [10]:
#Create a saver.
saver = tf.train.Saver()

#Launch the graph in a session, setup boilerplate
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    X, Y = inputs()
    total_loss = loss(X,Y)
    
    train_op = train(total_loss)
    coord = tf.train.Coordinator()
    
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    initial_step = 0
    
    #verify if we don't have a checkpoint saved already
    ckpt =tf.train.get_checkpoint_state(os.path.dirname('__file__'))
    if ckpt and ckpt.model_checkpoint_path:
        #Restores from checkpoint
        saver.restore(sess, ckpt.model_checkpoint_path)
        initial_step = int(ckpt.model_checkpoint_path.rsplit('-', 1)[1])
    
    # actual training loop
    training_steps = 100
    for step in range(initial_step, training_steps):
        sess.run([train_op])

        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:
            print "loss: ", sess.run([total_loss])
            saver.save(sess,'./my-model',global_step=step)

    evaluate(sess,X,Y)
    
    coord.request_stop()
    coord.join(threads)
    #save final model
    saver.save(sess, './my-model', global_step=training_steps)
    
    sess.close()

INFO:tensorflow:Restoring parameters from my-model-100
[[338.46683]]
[[282.56186]]
